# Progetto di ***Data Security and Privacy***

**Matteo Ghera** - matteo.ghera@stud.unifi.it

L'obiettivo di questo progetto è sviluppare un simulatore della macchina Enigma usata dalle truppe tedesche durante la Seconda Guerra Mondiale. Sotto riportiamo una breve descrizione tratta da [1]

In [1]:
#uso queste librerie per visualizzare le immagini

from IPython.display import Image
from functools import lru_cache

## Introduzione

Sotto è riportato lo schema contenente gli elementi elettromeccanici essenziali della macchina Enigma, tratto da [1].

In [2]:
Image(url='img/struttura_enigma.PNG')

Gli elementi elettromeccanici essenziali sono i seguenti:

- una *tastiera* che viene usata per inserire il testo cifrato o in chiaro;
- una *plugboard* che realizza uno scambio tra alcune coppie di lettere (tipicamente 10) attraverso una serie di connessioni. Ad esempio $Q \to S$ e $S \to Q$;
- tre *rotori* connessi a cascata. Ciascun rotore implementa una fissata permutazione tratta da un set di possibili permutazioni. Un rotore può essere fatto avanzare, rispetto ad una determinata posizione, di un numero di posizioni minore di 26. Più precisamente, il primo rotore (quello più a destra) avanza circolarmente di una posizione ad ogni lettera, il secondo rotore (rotore centrale) avanza di una posizione ogni 26 lettere mentre il terzo avanza di una posizioni ogni $26^2$ lettere;
- un *riflettore* che realizza uno scambio fissato tra 13 coppie di lettere (quindi ogni lettera uno scambio);
- una *lampboard* sulla quale viene visualizzato l'output (ciphertext o plaintext ottenuti), una lettera alla volta.

Come descritto in [2]. Supponiamo di aver fissato le permutazioni dei tre rotori. E\' possibile rappresentare la configurazione dei tre rotori in una tabella nel seguente modo: 

In [3]:
Image(url='img/Enigma.png')

Per ciascuna componente la lista di destra rappresenta la traformazione eseguita dal rotore mentre la lista di sinistra indica il circuito di uscita. Supponiamo di premere il tasto K sulla tastiera. Nella configurazione UOLY, la tensione attraversando il circuito arriverà alla lettera O del rotore destro; qui attraverserà la connessione che la collega alla lettera O sul lato opposto per proseguire quindi verso la lettera A del rotore centrale. Il procedimento è analogo fino a quando non si raggiuge il riflettore. Il riflettore riceverà la corrente attraverso il circuito relativo alla lettera J e seguendo il collegamento sul lato opposto raggiungerà il rotore sinistro alla lettera L. Il procedimento si ripete a ritroso. Sul lampboard si illuminerà la lettera J. La tabella sottostante raffigura il procedimento illustrato.    

In [4]:
Image(url='img/Enigma2.PNG')

## Descrizione implementazione

Utilizzando l'osservazione precedente ho provveduto ad implementare la classe Python *EnigmaRotor* che descrive il comportamento di un generico rotore. Il costruttore di tale classe riceve in ingresso: le due liste chiamate *Entrata* e *Uscita* che rappresentano la permutazione scelta per il rotore, il rotore successivo (quello a cui sarà inviato il segnale di rotazione una volta completato il giro) e due flag che indicano rispettivamente se il rotore corrente ha un successivo e se è ruotabile. Il metodo *impostaPosizione* imposta la configurazione iniziale del rotore. Data la posizione nel vettore *Entrata*, il metodo *posizioneSinistra* cerca la lettera corrispondente nel vettore *Uscita* e restituisce la sua posizione; per cercare la posizione nel vettore *Entrata* relativa alla lettera del vettore *Uscita* con posizione data si usa il metodo *posizioneDestra*. Qualore fosse consentito, il metodo *muovi* muove il rotore dato di una posizione (verso l'alto), invia un segnale al vettore successivo, se esiste.

La classe Python *EnigmaMachine* simula il comportamento di una macchina enigma. Il costruttore di tale classe riceve in ingresso quattro oggetti di tipo *EnigmaRotor*: *riflettore*, *rotoreSinistro*, *rotoreCentrale*, *rotoreDestro*; e due liste: *alfabeto* e *plugboard*. Si osservi che il riflettore è un tipo particolare di rotore che non ruota e che il rotore sinistro non ha alcun successore. L'alfabeto in input è quello riportato nella colonna "Iniziale" delle tabelle precendenti conosciuto come QWERTZ. Questa lista rappresenta la sequenza di lettere che si ottiene leggendo la tastiera della macchina Enigma da sinistra verso destra, dall'alto verso il basso. Per tutto il progetto utilizzeremo questa alfabeto e non quello convenzionale. La lista *plugboard* è una sequenza di numeri da 0 a 25; supponiamo di premere il tasto $i$ sulla tastiera allora la lettera realmente codificata sarà data da $plugboard[i]$. In questo modo sarà possibile eseguire lo scambio fissato indicato da ciascuna coppia di lettere.

Il metodo *impostaEnigma* sarà utilizzato per impostare le posizioni dei rotori e del riflettore mentre il metodo *esegui* cripta/decripta il messaggio in input.  

In [5]:
import main.EnigmaRotor
import main.EnigmaMachine

Ignoriamo per un momento la *plugboard*, possiamo inizializzare la macchina Enigma descritta dalle tabelle precedenti utilizzando il seguente codice Python:

In [6]:
#plugboard
plugboard=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]

#alfabeto
alfabeto=['Q', 'W', 'E', 'R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P', 'Y', 'X', 'C', 'V', 'B', 'N', 'M', 'L']

#riflettore
entrata=['R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P', 'Y', 'X', 'C', 'V', 'B', 'N', 'M', 'L', 'Q', 'W', 'E']
uscita= ['H', 'U', 'G', 'T', 'E', 'V', 'J', 'C', 'L', 'X', 'Z', 'R', 'A', 'Q', 'N', 'B', 'F', 'S', 'O', 'Y', 'P', 'W', 'D', 'K', 'M', 'I']
riflettore = main.EnigmaRotor.EnigmaRotor(entrata, uscita, None, False)

#rotore 3
entrata=['T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P','Y', 'X', 'C', 'V', 'B', 'N', 'M', 'L', 'Q', 'W', 'E', 'R']
uscita= ['F', 'M', 'Y', 'Z', 'E', 'Q', 'D', 'L', 'B', 'C', 'K', 'J', 'G', 'V','P', 'U', 'R', 'W', 'N', 'X', 'A', 'H', 'S', 'T', 'I', 'O']
rotoreSinistro = main.EnigmaRotor.EnigmaRotor(entrata, uscita)

# rotore 2
entrata =['L', 'Q', 'W', 'E', 'R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P', 'Y', 'X', 'C', 'V', 'B', 'N', 'M']
uscita = ['E', 'Y', 'U', 'F', 'H', 'X', 'Z', 'M', 'N', 'J', 'G', 'O', 'P', 'A', 'Q', 'I', 'R', 'L', 'D', 'T', 'W', 'V', 'K', 'S', 'B', 'C']
rotoreCentrale = main.EnigmaRotor.EnigmaRotor(entrata, uscita, rotoreSinistro)

# rotore 1
entrata = ['Y', 'X', 'C', 'V', 'B', 'N', 'M', 'L', 'Q', 'W', 'E', 'R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D',
           'F', 'G', 'H', 'J', 'K', 'P']
uscita = ['I', 'J', 'U', 'H', 'X', 'Z', 'M', 'N', 'A', 'V', 'O', 'E', 'Y', 'F', 'W', 'L', 'D', 'Q', 'C', 'B',
          'S', 'P', 'T', 'K', 'R', 'G']
rotoreDestro=main.EnigmaRotor.EnigmaRotor(entrata, uscita, rotoreCentrale)

enigma=main.EnigmaMachine.EnigmaMachine(riflettore, rotoreSinistro, rotoreCentrale, rotoreDestro, alfabeto, plugboard)
enigma.impostaEnigma('U','O','L','Y')


In [7]:
enigma.esegui('K')

'J'

Possiamo verificare che se si imposta la macchina Enigma con la stessa configurazione iniziale e si decripta 'J' si ottiene 'K'. Infatti abbiamo:

In [8]:
enigma.impostaEnigma('U','O','L','Y')
enigma.esegui('J')

'K'

Se proviamo a criptare il seguente frammento di plaintext WETTERVORHERSAGEBISKAYA (previsioni del tempo Biscaglia) con l'attuale configurazione della macchina si ottiene:

In [9]:
enigma.impostaEnigma('U','O','L','Y')
enigma.esegui('WETTERVORHERSAGEBISKAYA')

'YGSLMGTJHUDDJTJVTYBSFTR'

In [10]:
enigma.impostaEnigma('U','O','L','Y')
enigma.esegui('YGSLMGTJHUDDJTJVTYBSFTR')

'WETTERVORHERSAGEBISKAYA'

In [11]:
enigma.impostaEnigma('U','O','L','Y')
enigma.esegui('A')

'H'

Supponiamo di voler utilizzare la plugboard per trasformare la A in K e viceversa. Configuriamo di nuovo la macchina Enigma. Nel nostro alfabeto la lettera A è in nona posizione mentre la lettera K è sedicesima. Si imposta la nona posizione della lista *plugboard* a 16 e la sedicesima a 9. 

In [12]:
# trasformo A in K e viceversa
# plugboard
plugboard = [0, 1, 2, 3, 4, 5, 6, 7, 8, 16, 10, 11, 12, 13, 14, 15, 9, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

# alfabeto
alfabeto = ['Q', 'W', 'E', 'R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P', 'Y', 'X',
            'C', 'V', 'B', 'N', 'M', 'L']

# riflettore
entrata = ['R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P', 'Y', 'X', 'C', 'V', 'B',
           'N', 'M', 'L', 'Q', 'W', 'E']
uscita = ['H', 'U', 'G', 'T', 'E', 'V', 'J', 'C', 'L', 'X', 'Z', 'R', 'A', 'Q', 'N', 'B', 'F', 'S', 'O', 'Y',
          'P', 'W', 'D', 'K', 'M', 'I']
riflettore = main.EnigmaRotor.EnigmaRotor(entrata, uscita, None, False)

# rotore 3
entrata = ['T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P', 'Y', 'X', 'C', 'V', 'B', 'N',
           'M', 'L', 'Q', 'W', 'E', 'R']
uscita = ['F', 'M', 'Y', 'Z', 'E', 'Q', 'D', 'L', 'B', 'C', 'K', 'J', 'G', 'V', 'P', 'U', 'R', 'W', 'N', 'X',
          'A', 'H', 'S', 'T', 'I', 'O']
rotoreSinistro = main.EnigmaRotor.EnigmaRotor(entrata, uscita)

# rotore 2
entrata = ['L', 'Q', 'W', 'E', 'R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'P', 'Y',
           'X', 'C', 'V', 'B', 'N', 'M']
uscita = ['E', 'Y', 'U', 'F', 'H', 'X', 'Z', 'M', 'N', 'J', 'G', 'O', 'P', 'A', 'Q', 'I', 'R', 'L', 'D', 'T',
          'W', 'V', 'K', 'S', 'B', 'C']
rotoreCentrale = main.EnigmaRotor.EnigmaRotor(entrata, uscita, rotoreSinistro)

# rotore 1
entrata = ['Y', 'X', 'C', 'V', 'B', 'N', 'M', 'L', 'Q', 'W', 'E', 'R', 'T', 'Z', 'U', 'I', 'O', 'A', 'S', 'D',
           'F', 'G', 'H', 'J', 'K', 'P']
uscita = ['I', 'J', 'U', 'H', 'X', 'Z', 'M', 'N', 'A', 'V', 'O', 'E', 'Y', 'F', 'W', 'L', 'D', 'Q', 'C', 'B',
          'S', 'P', 'T', 'K', 'R', 'G']
rotoreDestro = main.EnigmaRotor.EnigmaRotor(entrata, uscita, rotoreCentrale)

enigma = main.EnigmaMachine.EnigmaMachine(riflettore, rotoreSinistro, rotoreCentrale, rotoreDestro,
                                               alfabeto, plugboard)
enigma.impostaEnigma('U', 'O', 'L', 'Y')

Proviamo a codificare la lettera 'K', la sua codifica deve coincidere con quella della lettera 'A' calcolata in In[11] con la macchina Enigma precedente che non usava la pluboard.

In [14]:
enigma.impostaEnigma('U', 'O', 'L', 'Y')
enigma.esegui('K')

'H'

In [15]:
enigma.impostaEnigma('U', 'O', 'L', 'Y')
enigma.esegui('H')

'K'

Si osservi che si può utilizzare questo simulatore di macchina Enigma con qualsiasi alfabeto e qualsiasi permutazione dei rotori e del riflettore che realizzino involuzioni.

## Riferimenti

[1] Davide Salamon, *Data Privacy and Security*, Springer, 2003  

[2] http://www.crittologia.eu/